## Building a BATTLE Eval

In [21]:
file_name = "dataset"
model = "gpt-3.5-turbo"
choices = "user"                     # options: user, ai
num_few_shot = 0
initial_message = [{"role": "user", "content": ""}]
roles = "user"                      # options: system-user, system, user
battles_generation = "complete-chat"        # options: dense, only-new, complete-chat

#!cd evals
#!git lfs fetch --all
#!git lfs pull

In [22]:
import os
from openai import OpenAI
import pandas as pd
import yaml
from IPython.display import display_markdown
import copy

client = OpenAI()
#import datetime
#from dotenv import load_dotenv

#load_dotenv()  # Load environment variables from .env file

#api_key = os.environ.get("OPENAI_API_KEY")

# Install Evals if you haven't already
# %pip install -e ../.
# pip install --upgrade openai
# %pip install pandas


In [23]:
# Paths. Assuming this notebook is in examples/

evals_path = os.path.join(os.getcwd(), "..", "evals")

registry_path = os.path.join(evals_path, "registry", "evals", "battles.yaml")

data_path = os.path.join(evals_path, "registry", "data", "battles")
os.makedirs(data_path, exist_ok=True)
data_path = os.path.join(data_path, "samples.jsonl")

json_logs_path = os.path.join(os.getcwd(), "logs")
os.makedirs(json_logs_path, exist_ok=True)
json_logs_path = os.path.join(json_logs_path, "logs")

df_path = os.path.join(evals_path, "evallogs", "df")
os.makedirs(df_path, exist_ok=True)
dataset_path = os.path.join(df_path, file_name)

In [24]:
# Registry yaml

registry_yaml = {}

registry_yaml["battles"] = {
    "id": "battles.test.v1",
    "metrics": ["accuracy"]
}
registry_yaml["battles.test.v1"] = {
    "class": "evals.elsuite.modelgraded.classify:ModelBasedClassify",
    "args": {
        "samples_jsonl": "battles/samples.jsonl",
        "eval_type": "cot_classify",
        "modelgraded_spec": "battle"
    }
}
with open(os.path.join(registry_path), "w") as f:
    yaml.dump(registry_yaml, f)

In [25]:
def best(dataset, memory_limit=False):
    #best_message = dataset[dataset["Choice"]=="Yes"].iloc[-1]["Instruction1"]
    #best_response = dataset[dataset["Choice"]=="Yes"].iloc[-1]["Response1"]
    if dataset.iloc[-1]["Choice"] == "Yes":
        best_content = dataset.iloc[-1]["Instruction1"]
        best_response = dataset.iloc[-1]["Response1"]
    else:
        best_content = dataset.iloc[-1]["Instruction2"]
        best_response = dataset.iloc[-1]["Response2"]
    if memory_limit:
        best_message = [{"role": "user", "content": best_content}]
    if not memory_limit: 
        try:
            best_message = dataset[dataset["Choice"]=="Yes"].iloc[-1]["Data"]["Full message"]
        except IndexError:
            best_message = dataset.iloc[0]["Data"]["Full message"]
    return best_message,best_response

In [42]:
# Load data

try:
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    best_message, best_response = best(dataset)

# Or create new data file

except FileNotFoundError:
    completion = client.chat.completions.create(
        messages = initial_message,
        model = model,
        )
    dataset = pd.DataFrame(
        [
            ["", "", "", "", "", "", {}],
            ["", completion.choices[0].message.content, "", "", "", "", {}]
            ], 
        columns=['Instruction1', 'Response1', 'Instruction2', 'Response2', 'Sampled', 'Choice', 'Data']
    ) 
    dataset.at[0, "Data"] = {"Completion":completion.to_dict(),"Full message":initial_message}
candidate_messages = [initial_message]
next_candidate_messages = []
generation_distance = 0
message_in_candidate_messages = 0
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")


In [27]:
dataset.iloc[0]["Data"]

{'Completion': {'id': 'chatcmpl-9vlyBUsCsX4wC8ifcEgQzh1NHjsH9',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': 'Hello! How can I assist you today?',
     'role': 'assistant',
     'refusal': None}}],
  'created': 1723556119,
  'model': 'gpt-3.5-turbo-0125',
  'object': 'chat.completion',
  'system_fingerprint': None,
  'usage': {'completion_tokens': 9, 'prompt_tokens': 7, 'total_tokens': 16}},
 'Full message': [{'role': 'user', 'content': ''}]}

In [28]:
def battle(best_message, best_response, candidate_message):

    dataset = [{"input1": candidate_message, "input2": best_message, "completion2":best_response}]

    df = pd.DataFrame(dataset)
    df.to_json(data_path, orient="records", lines=True)

    !oaieval gpt-3.5-turbo battles --record_path logs/logs
    
    with open(json_logs_path, "r") as f:
        df = pd.read_json(f, lines=True)

    #current_time = datetime.datetime.now()
    #formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
    #df.to_json(os.path.join(df_path, formatted_time), lines=True, orient="records")

    instruction1 = candidate_message[-1]["content"]
    instruction2 = best_message[-1]["content"]

    battle_prompt_content = df["data"].iloc[-2]["prompt"][0]["content"]
    response1 = battle_prompt_content.split("\n[Response 1]\n",)[1].split("\n\n[Instruction 2]\n")[0]
    response1 = response1.replace("\\'", "'").replace("\\n", "\n")
    response2 = battle_prompt_content.split("\n[Response 2]\n",)[1].split("\n\n\nIs the first response better than the second?")[0]
    response2 = response2.replace("\\'", "'").replace("\\n", "\n")
    print(f"Response1: {response1}")
    #print(f"response2: {response2}")

    sampled = df["data"].iloc[-2]["sampled"][0]

    choice = df["data"].iloc[-1]["choice"]

    data = {'Instruction1': instruction1, 'Response1': response1, 'Instruction2': instruction2, 'Response2': response2, 'Sampled': sampled, 'Choice': choice, 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = df.to_dict()
    return data

In [29]:
def switch_system_and_user(message):
    new_message = []
    if message[0]["role"] == "system":
        new_message.append([{"role": "user", "content": message[0]["content"]}])
    else:
        new_message.append([{"role": "system", "content": message[0]["content"]}])
    return new_message

In [30]:
def find_parents(content, dataset):
    #print (f"finding parents {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Response1"] == content]
    parents = dataset["Instruction1"].unique()
    return parents

In [31]:
def find_children(content, dataset):
    #print (f"finding children {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Instruction1"] == content]
    children = dataset["Response1"].unique()
    return children

In [32]:
def list_candidate_messages(dataset,generation_distance,roles="user"):
    best_message = best(dataset)[0]
    best_content = best_message[-1]["content"]
    last = [best_content]
    list_of_contents = [best_content]

    next = []
    for i in range(generation_distance):
        #print (f"starting level {i}, generation distance {generation_distance}")
        for content in last:
            #print(f"last_up: {last_up}, now {content}")
            next.extend(find_parents(content, dataset))
            next.extend(find_children(content, dataset))
            
        #print (f"behind for loops")
        list_of_contents.extend(next)
        last = next.copy()
        #print(f"endind level {i}")
        
    list_of_contents = pd.array(list_of_contents).unique().tolist()
    
    #best_message_index = list_of_contents.index(best_message[0][content])
    #start = max(best_message_index - generation_distance, 0)
    #stop = min(len(list_of_contents))
    #stop = len(list_of_contents) - 1

    messages = []
    for content in list_of_contents:
        if roles == "system-user" or "user":
            messages.append([{"role":"user","content":content}])
        if (roles == "system-user") or (roles == "system"):
            print("Creating message without user role.")
            messages.append([{"role":"system","content":content}])

    return messages

In [33]:
def make_x_shot_prompt(dataset,message,num_few_shot):
    few_shot_prompt = []
    for _, key_battle in dataset[dataset["Choice"]=="Yes"].iloc[-num_few_shot:].iterrows():
        few_shot_prompt.append({"role": "system", "content": key_battle["Instruction1"], "name": "example_user"})
        few_shot_prompt.append({"role": "system", "content": key_battle["Response1"], "name": "example_assistant"})
    message = few_shot_prompt + message
    return message

In [34]:
def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages

In [35]:
def user_choice_prompt(candidate_message,n,best_message,best_response):

    prompt = "You are comparing two responses to the following two instructions."

    prompt += "\n\n[Instruction 1]\n"
    prompt += candidate_message[-1]["content"]

    prompt += "\n\n[Response 1]\n"
    completion = client.chat.completions.create(
        messages = candidate_message,
        model = model,
        )
    response1 = completion.choices[0].message.content
    prompt += response1
    
    prompt += "\n\n[Instruction 2]\n"
    best_content = best_message[-1]["content"]
    prompt += best_content

    prompt += "\n\n[Response 2]\n"
    prompt += best_response

    prompt += "\n\n\nIs the first response better than the second? You must provide one answer based on your subjective view."

    data = {'Instruction1': candidate_message[-1]["content"], 'Response1': response1, 'Instruction2': best_content, 'Response2': best_response, 'Sampled': "", 'Choice': "", 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict(),"Full message":candidate_message,"Parent":n}

    display_markdown(prompt, raw=True)
    return data,candidate_messages,message_in_candidate_messages,generation_distance

In [36]:
def user_choice_prompt_old(candidate_messages,message_in_candidate_messages,dataset, generation_distance, roles, battles_generation,num_few_shot):

    prompt = "You are comparing two responses to the following two instructions."

    prompt += "\n\n[Instruction 1]\n"
    if battles_generation == "dense":
        if message_in_candidate_messages < len(candidate_messages):
            candidate_message = candidate_messages[message_in_candidate_messages]
            message_in_candidate_messages += 1
        else:
            candidate_messages = list_candidate_messages(dataset, generation_distance, roles)
            generation_distance += 1
            candidate_message = candidate_messages[0]
            message_in_candidate_messages = 1
            print(f"candidate messages listed, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")
    if battles_generation == "complete-chat":
        chat = best(dataset)[0] + [{'role': 'assistant', 'content': best(dataset)[1]}]
        candidate_message = transform_roles(chat)
    if num_few_shot > 0:
        candidate_message = make_x_shot_prompt(dataset,candidate_message,num_few_shot)
    prompt += candidate_message[-1]["content"]

    prompt += "\n\n[Response 1]\n"
    completion = client.chat.completions.create(
        messages = candidate_message,
        model = model,
        )
    response1 = completion.choices[0].message.content
    prompt += response1
    
    prompt += "\n\n[Instruction 2]\n"
    best_content = best(dataset)[0][-1]["content"]
    prompt += best_content

    prompt += "\n\n[Response 2]\n"
    prompt += best(dataset)[1]

    prompt += "\n\n\nIs the first response better than the second? You must provide one answer based on your subjective view."

    data = {'Instruction1': candidate_message[-1]["content"], 'Response1': response1, 'Instruction2': best_content, 'Response2': best(dataset)[1], 'Sampled': "", 'Choice': "", 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict(),"Full message":candidate_message}

    display_markdown(prompt, raw=True)
    return data,candidate_messages,message_in_candidate_messages,generation_distance

In [37]:
"""if choices == "user":
    data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt_old(
        candidate_messages,
        message_in_candidate_messages,
        dataset,
        generation_distance,
        roles,
        battles_generation,
        num_few_shot)"""

'if choices == "user":\n    data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt_old(\n        candidate_messages,\n        message_in_candidate_messages,\n        dataset,\n        generation_distance,\n        roles,\n        battles_generation,\n        num_few_shot)'

In [44]:
def continue_chat(n, dataset_path):
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    last_response = dataset.iloc[n]["Response1"]
    chat = dataset.iloc[n]["Data"]["Full message"] + [{'role': 'assistant', 'content': last_response}]
    candidate_message = transform_roles(chat)
    last_response = dataset.iloc[n]["Response1"]
    parent = n
    data = user_choice_prompt(candidate_message,parent,candidate_message,last_response)[0]
    return data

In [38]:
def improve_chat(n, dataset_path):
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    candidate_message = dataset.iloc[n]["Data"]["Full message"]
    last_response = dataset.iloc[n]["Response1"]
    parent = dataset.iloc[n]["Data"]["Parent"]
    data = user_choice_prompt(candidate_message,parent,candidate_message,last_response)[0]
    return data

In [136]:
dataset

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,Hello! How can I assist you today?,,,,,{'Completion': {'id': 'chatcmpl-9vm1k6DTcedps9...
1,Hello! How can I assist you today?,Hello! I'm here to help you with any questions...,Hello! How can I assist you today?,Hello! How can I assist you today?,,Yes,{'Completion': {'id': 'chatcmpl-9vm3VtOJSDLdkZ...
2,Hello! I'm here to help you with any questions...,Thank you for offering your help! Is there any...,Hello! I'm here to help you with any questions...,Hello! I'm here to help you with any questions...,,Yes,{'Completion': {'id': 'chatcmpl-9vmC68rQXaRrQi...
3,Thank you for offering your help! Is there any...,Thank you for asking! I would be happy to talk...,Thank you for offering your help! Is there any...,Thank you for offering your help! Is there any...,,Yes,{'Completion': {'id': 'chatcmpl-9vmP8gIYZm4IGv...
4,Thank you for asking! I would be happy to talk...,That sounds like a wide range of interesting t...,Thank you for asking! I would be happy to talk...,Thank you for asking! I would be happy to talk...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQ4A7bsHwEYN...
5,That sounds like a wide range of interesting t...,Thank you for your willingness to engage in di...,That sounds like a wide range of interesting t...,That sounds like a wide range of interesting t...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQgrivqLk0Ew...
6,Thank you for your willingness to engage in di...,Certainly! One of the most notable technology ...,Thank you for your willingness to engage in di...,Thank you for your willingness to engage in di...,,Yes,{'Completion': {'id': 'chatcmpl-9vmRZnCMqSiQE4...
7,Certainly! One of the most notable technology ...,Those are indeed fascinating developments! Art...,Certainly! One of the most notable technology ...,Certainly! One of the most notable technology ...,,Yes,{'Completion': {'id': 'chatcmpl-9vmSbrALb8i8z5...
8,Those are indeed fascinating developments! Art...,I find all of these technology trends to be in...,Those are indeed fascinating developments! Art...,Those are indeed fascinating developments! Art...,,Yes,{'Completion': {'id': 'chatcmpl-9vmUIDaLrSgMhP...
9,I find all of these technology trends to be in...,Thank you for your willingness to delve deeper...,I find all of these technology trends to be in...,I find all of these technology trends to be in...,,Yes,{'Completion': {'id': 'chatcmpl-9vmWB2qIjxtDXN...


In [137]:
n = 0
data = improve_chat(n,dataset_path)

KeyError: 'Parent'

In [133]:
choice = "No"
choice = "Yes"
data.at[0, "Choice"] = choice
print(f"Your answer:{choice}")
if choice == "Yes":
    print(f"New best message and response!")
    if battles_generation == "dense":
        generation_distance = 0
dataset = pd.concat([dataset, data],ignore_index=True)
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
#data,candidate_messages,message_in_candidate_messages,generation_distance = user_choice_prompt_old(candidate_messages,message_in_candidate_messages,dataset,generation_distance,roles,battles_generation,num_few_shot)

Your answer:Yes
New best message and response!


In [148]:
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")

In [ ]:
def is_parent(parent,response,child,):
    if parent["content"] == "":
        parent = []
    lp = len(parent)
    lch = len(child)
    if lp != lch + 1:
        return False
    if msg_to_content(child)[:l+1] != msg_to_content(parent).append(response):
        return False
    else:
        return True

In [ ]:
def msg_to_content(msg):
    content = []
    for line in msg:
        content.append(line["content"])
    return content

In [ ]:
def best_msg_level_up(dataset, msg, best_chat):
    level = 0 if best_chat.empty else len(msg)
    filtered_dataset = dataset[
        (dataset["Choice"] != "No") & 
        (dataset["Data"].apply(lambda x: len(x["Full message"])) == level + 1) &
        (dataset["Data"].apply(lambda x: is_parent(x["Full message"], msg, dataset)))
        ] 
    if len(filtered_dataset) > 1:
        #print(filtered_dataset)
        filtered_dataset = filtered_dataset.iloc[[-1]]
        print(filtered_dataset)
    if filtered_dataset.empty:
        return best_chat
    best_chat = pd.concat([best_chat, filtered_dataset],ignore_index=True)
    #try:
    #print(filtered_dataset.iloc[0])
    msg = filtered_dataset.iloc[0]["Data"]["Full message"]
    #except:
        #print("Returning filtered dataset")
        #return filtered_dataset.iloc[0]
    best_msg_level_up(dataset, msg, best_chat)
    return best_chat
    

In [ ]:
best_chat = best_msg_level_up(dataset, initial_message, pd.DataFrame([],columns=dataset.columns))

In [ ]:
best_chat["Data"] = best_chat.apply(lambda x: len(x["Data"]))


In [ ]:
best_chat = pd.DataFrame([],columns=dataset.columns)
msg = initial_message




In [ ]:
level = 0 if best_chat.empty else len(msg)
filtered_dataset = dataset[
    (dataset["Choice"] != "No") & 
    (dataset["Data"].apply(lambda x: len(x["Full message"])) == level + 1) &
    (dataset.apply(lambda x: is_parent(msg,x["Data"]["Full message"],x["Response1"])))
    ] 
if len(filtered_dataset) > 1:
    #print(filtered_dataset)
    filtered_dataset = filtered_dataset.iloc[[-1]]
    print(filtered_dataset)
#if filtered_dataset.empty:
    #return best_chat
best_chat = pd.concat([best_chat, filtered_dataset])
#try:
#print(filtered_dataset.iloc[0])
msg = filtered_dataset.iloc[0]["Data"]["Full message"]
#except:
    #print("Returning filtered dataset")
    #return filtered_dataset.iloc[0]
#best_msg_level_up(dataset, msg, best_chat)

In [139]:
first_row = pd.DataFrame(
    [
        ["", "", "", "", "", "", {}]
        ], 
    columns=['Instruction1', 'Response1', 'Instruction2', 'Response2', 'Sampled', 'Choice', 'Data']
) 

In [142]:
dataset

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,Hello! How can I assist you today?,,,,,{'Completion': {'id': 'chatcmpl-9vm1k6DTcedps9...
1,Hello! How can I assist you today?,Hello! I'm here to help you with any questions...,Hello! How can I assist you today?,Hello! How can I assist you today?,,Yes,{'Completion': {'id': 'chatcmpl-9vm3VtOJSDLdkZ...
2,Hello! I'm here to help you with any questions...,Thank you for offering your help! Is there any...,Hello! I'm here to help you with any questions...,Hello! I'm here to help you with any questions...,,Yes,{'Completion': {'id': 'chatcmpl-9vmC68rQXaRrQi...
3,Thank you for offering your help! Is there any...,Thank you for asking! I would be happy to talk...,Thank you for offering your help! Is there any...,Thank you for offering your help! Is there any...,,Yes,{'Completion': {'id': 'chatcmpl-9vmP8gIYZm4IGv...
4,Thank you for asking! I would be happy to talk...,That sounds like a wide range of interesting t...,Thank you for asking! I would be happy to talk...,Thank you for asking! I would be happy to talk...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQ4A7bsHwEYN...
5,That sounds like a wide range of interesting t...,Thank you for your willingness to engage in di...,That sounds like a wide range of interesting t...,That sounds like a wide range of interesting t...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQgrivqLk0Ew...
6,Thank you for your willingness to engage in di...,Certainly! One of the most notable technology ...,Thank you for your willingness to engage in di...,Thank you for your willingness to engage in di...,,Yes,{'Completion': {'id': 'chatcmpl-9vmRZnCMqSiQE4...
7,Certainly! One of the most notable technology ...,Those are indeed fascinating developments! Art...,Certainly! One of the most notable technology ...,Certainly! One of the most notable technology ...,,Yes,{'Completion': {'id': 'chatcmpl-9vmSbrALb8i8z5...
8,Those are indeed fascinating developments! Art...,I find all of these technology trends to be in...,Those are indeed fascinating developments! Art...,Those are indeed fascinating developments! Art...,,Yes,{'Completion': {'id': 'chatcmpl-9vmUIDaLrSgMhP...
9,I find all of these technology trends to be in...,Thank you for your willingness to delve deeper...,I find all of these technology trends to be in...,I find all of these technology trends to be in...,,Yes,{'Completion': {'id': 'chatcmpl-9vmWB2qIjxtDXN...


In [144]:
dataset = pd.concat([first_row, dataset],ignore_index=True)

In [146]:
dataset = dataset.drop([21])

In [147]:
dataset

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,,,,,,{}
1,,Hello! How can I assist you today?,,,,,{'Completion': {'id': 'chatcmpl-9vm1k6DTcedps9...
2,Hello! How can I assist you today?,Hello! I'm here to help you with any questions...,Hello! How can I assist you today?,Hello! How can I assist you today?,,Yes,{'Completion': {'id': 'chatcmpl-9vm3VtOJSDLdkZ...
3,Hello! I'm here to help you with any questions...,Thank you for offering your help! Is there any...,Hello! I'm here to help you with any questions...,Hello! I'm here to help you with any questions...,,Yes,{'Completion': {'id': 'chatcmpl-9vmC68rQXaRrQi...
4,Thank you for offering your help! Is there any...,Thank you for asking! I would be happy to talk...,Thank you for offering your help! Is there any...,Thank you for offering your help! Is there any...,,Yes,{'Completion': {'id': 'chatcmpl-9vmP8gIYZm4IGv...
5,Thank you for asking! I would be happy to talk...,That sounds like a wide range of interesting t...,Thank you for asking! I would be happy to talk...,Thank you for asking! I would be happy to talk...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQ4A7bsHwEYN...
6,That sounds like a wide range of interesting t...,Thank you for your willingness to engage in di...,That sounds like a wide range of interesting t...,That sounds like a wide range of interesting t...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQgrivqLk0Ew...
7,Thank you for your willingness to engage in di...,Certainly! One of the most notable technology ...,Thank you for your willingness to engage in di...,Thank you for your willingness to engage in di...,,Yes,{'Completion': {'id': 'chatcmpl-9vmRZnCMqSiQE4...
8,Certainly! One of the most notable technology ...,Those are indeed fascinating developments! Art...,Certainly! One of the most notable technology ...,Certainly! One of the most notable technology ...,,Yes,{'Completion': {'id': 'chatcmpl-9vmSbrALb8i8z5...
9,Those are indeed fascinating developments! Art...,I find all of these technology trends to be in...,Those are indeed fascinating developments! Art...,Those are indeed fascinating developments! Art...,,Yes,{'Completion': {'id': 'chatcmpl-9vmUIDaLrSgMhP...


In [ ]:
if choices == "ai":
    for candidate_message in candidate_messages:
        if num_few_shot > 0:
            candidate_message = make_x_shot_prompt(dataset,candidate_message,num_few_shot)
        data = battle(best_message, best_response, candidate_message)
        dataset = pd.concat([dataset, data],ignore_index=True)
        dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
        if data["Choice"].iloc[0] == "Yes":
            best_message = candidate_message
            best_response = data["Response1"].iloc[0]
            print(f"New best message:{best_message}")
            if battles_generation == "dense":
                generation_distance = 0

        print(f"Instruction1: {candidate_message[-1]["content"]}")

        if battles_generation == "only-new":
            new_message = [{"role": candidate_message[0]["role"], "content":data["Response1"].iloc[0]}]
            if new_message not in next_candidate_messages:
                next_candidate_messages.append(new_message)
                if roles == "system-user":
                    next_candidate_messages.append(switch_system_and_user(new_message))

    if battles_generation == "only-new":
        candidate_messages = next_candidate_messages
        next_candidate_messages = []
    if battles_generation == "dense":
        #print("going to list candidate messages")
        candidate_messages = list_candidate_messages(dataset, best_message, generation_distance, roles)
        generation_distance += 1
        #print("candidate messages listed")
    print(f"all done, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")

In [94]:
dataset

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,Hello! How can I assist you today?,,,,,{'Completion': {'id': 'chatcmpl-9vm1k6DTcedps9...
1,Hello! How can I assist you today?,Hello! I'm here to help you with any questions...,Hello! How can I assist you today?,Hello! How can I assist you today?,,Yes,{'Completion': {'id': 'chatcmpl-9vm3VtOJSDLdkZ...
2,Hello! I'm here to help you with any questions...,Thank you for offering your help! Is there any...,Hello! I'm here to help you with any questions...,Hello! I'm here to help you with any questions...,,Yes,{'Completion': {'id': 'chatcmpl-9vmC68rQXaRrQi...
3,Thank you for offering your help! Is there any...,Thank you for asking! I would be happy to talk...,Thank you for offering your help! Is there any...,Thank you for offering your help! Is there any...,,Yes,{'Completion': {'id': 'chatcmpl-9vmP8gIYZm4IGv...
4,Thank you for asking! I would be happy to talk...,That sounds like a wide range of interesting t...,Thank you for asking! I would be happy to talk...,Thank you for asking! I would be happy to talk...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQ4A7bsHwEYN...
5,That sounds like a wide range of interesting t...,Thank you for your willingness to engage in di...,That sounds like a wide range of interesting t...,That sounds like a wide range of interesting t...,,Yes,{'Completion': {'id': 'chatcmpl-9vmQgrivqLk0Ew...
6,Thank you for your willingness to engage in di...,Certainly! One of the most notable technology ...,Thank you for your willingness to engage in di...,Thank you for your willingness to engage in di...,,Yes,{'Completion': {'id': 'chatcmpl-9vmRZnCMqSiQE4...
7,Certainly! One of the most notable technology ...,Those are indeed fascinating developments! Art...,Certainly! One of the most notable technology ...,Certainly! One of the most notable technology ...,,Yes,{'Completion': {'id': 'chatcmpl-9vmSbrALb8i8z5...
8,Those are indeed fascinating developments! Art...,I find all of these technology trends to be in...,Those are indeed fascinating developments! Art...,Those are indeed fascinating developments! Art...,,Yes,{'Completion': {'id': 'chatcmpl-9vmUIDaLrSgMhP...
9,I find all of these technology trends to be in...,Thank you for your willingness to delve deeper...,I find all of these technology trends to be in...,I find all of these technology trends to be in...,,Yes,{'Completion': {'id': 'chatcmpl-9vmWB2qIjxtDXN...


In [101]:
for entry in dataset["Response1"]:
    display_markdown(entry, raw=True)

Hello! How can I assist you today?

Hello! I'm here to help you with any questions you might have or provide information on a variety of topics. Feel free to ask me anything!

Thank you for offering your help! Is there anything specific you would like to know more about or discuss today?

Thank you for asking! I would be happy to talk about various topics, such as technology, science, history, language learning, travel, and more. If you have any specific questions or interests, feel free to share them!

That sounds like a wide range of interesting topics to explore! If you have any questions or if you would like to delve deeper into any of these subjects, please let me know. I'm here to provide information and engage in discussions with you.

...

Certainly! One of the most notable technology trends in recent years is the rapid advancement of artificial intelligence (AI) and machine learning. AI is being integrated into various industries, including healthcare, finance, transportation, and more, to improve efficiency, productivity, and decision-making processes.

Another significant trend is the rise of 5G technology, which promises faster and more reliable wireless connectivity. This development is expected to have a profound impact on the Internet of Things (IoT), autonomous vehicles, smart cities, and other connected devices.

Additionally, advancements in quantum computing, augmented reality (AR) and virtual reality (VR) technology, cybersecurity, and sustainable energy solutions are also shaping the future of technology.

These trends are not only fascinating but also have the potential to transform the way we live, work, and interact with the world around us.

Those are indeed fascinating developments! Artificial intelligence, 5G technology, quantum computing, AR/VR, cybersecurity, and sustainable energy solutions are all key areas that are driving innovation and shaping the future of technology. Each of these trends offers exciting opportunities and challenges that will likely have a significant impact on society, economy, and beyond.

Do you have a particular interest in any of these technology trends, or is there a specific aspect you would like to discuss further?

I find all of these technology trends to be incredibly intriguing and important for the future. If you would like to delve deeper into any of these areas, I would be happy to provide more information or discuss specific aspects of interest.

Whether you have questions about how these technologies work, their potential applications, current challenges, or future implications, feel free to let me know. I'm here to help facilitate a deeper understanding of these exciting developments in the world of technology.

Thank you for your willingness to delve deeper into these technology trends! Let's explore the topic of artificial intelligence (AI) further. AI has made significant strides in recent years, with applications ranging from virtual assistants like Siri and Alexa to autonomous vehicles, medical diagnosis, and personalized recommendations.

One aspect of AI that is particularly fascinating is deep learning, a subset of machine learning that uses neural networks to model complex patterns and make decisions based on data. Deep learning has shown great promise in areas such as computer vision, natural language processing, and voice recognition.

Do you have any specific questions about AI or deep learning that you would like to discuss further? Or is there another aspect of technology that you are curious about exploring in more detail?

I'm glad to hear that you're interested in exploring more about artificial intelligence and deep learning! These are indeed fascinating areas of technology that have the potential to revolutionize various industries and vastly improve the way we interact with technology.

If you would like to delve deeper into the principles of deep learning, how neural networks work, the current applications of AI in different fields, or any specific examples of AI technology, I would be more than happy to provide more information and engage in a discussion on these topics.

Additionally, if there are any other areas of technology or specific topics you would like to explore, please feel free to let me know, and we can continue our discussion from there. I'm here to help answer your questions and provide insights on a wide range of subjects.

In [ ]:
test = dataset[dataset["Choice"]=="Yes"]#.iloc[0]
for row in test.iterrows():
        display_markdown(row[0], raw=True)
        display_markdown(row[1].iloc[1], raw=True)
